<a href="https://colab.research.google.com/github/maelmitc/ProjetIA-Ma-l-Manon/blob/main/Project/coucou%20manon%20hehe/RAG_Projectbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [3]:
from google.colab import files

uploaded = files.upload()

Saving 1d-monopoly-regle.pdf to 1d-monopoly-regle.pdf


In [4]:
file_name = list(uploaded.keys())[0]
file_content = uploaded[file_name]

In [5]:
import PyPDF2
import io


def extract_text_from_pdf_content(pdf_content):
    text = ""
    # Convert the file content into a file-like object
    pdf_file = io.BytesIO(pdf_content)
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    # Iterate through each page and extract text
    num_pages = len(pdf_reader.pages)
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

# Extract text from the uploaded PDF content
text = extract_text_from_pdf_content(file_content)
print(text)

Règles du jeu officielles du Monopoly®  
 
Garanties presque sans erreurs   
BUT DU JEU  :  
Etre le dernier joueur à rester en jeu, c'est -à-dire le dernier 
joueur n'ayant pas fait faillite.  
 
PREPARATION  
1. Prenez le casier de la Banque et disposez dans leurs emplacements les Maisons, les Hôtels, les Titre s de 
Propriété et les billets (classés selon leur valeur).   
2. La première fois que vous jouez, détachez les pieds en caoutchouc et disposez -les sous les coins des 
sabots.  
Disposez les sabots à l'endroit prévu sur le plateau de jeu.   
3. Détachez les cartes Chanc e et Caisse de Communauté, mélangez -les et placez -les face cachée dans les 
sabots correspondants.   
4. Chaque joueur choisit un pion et le place sur la case "Départ".  
5. Le Banquier et la Banque.  
Choisissez comme Banquier un joueur. S'il y a plus de 5 joueurs, l 'un des joueurs peut décider de tenir 
uniquement le rôle de Banquier.  
Le Banquier remet à chaque joueur une somme de 150.000 Francs, répart

##CHUNK  
First we will try to chunk by paragraph

First we clean the data by removing every words in Upper case letter that are not title to allow us to split the doc in chunks.

In [6]:
text_clean = text.replace("VOUS ETES LIBERE DE PRISON", "Vous etes libere de prison") \
                 .replace("ALLEZ EN PRISON", "allez en prison") \
                 .replace("CHANCE", "Chance")\
                 .replace("IMPOTS","impots")\
                 .replace("GARE","gare")\
                 .replace("LE VAINQUEUR", "le vainqueur")\
                 .replace("PARC GRATUIT","parc gratuit")\
                 .replace("TAXE DE LUXE","taxe de luxe")\
                 .replace("COMPAGNIE DE DISTRIBUTION", "compagnie de distribution")\
                 .replace("COMMUNAUTE","communaute")\
                 .replace("VOUS ETES", "vous etes") \
                 .replace("LIBERE DE PRISON","libere de prison")\
                 .replace("FIN DE LA PARTIE","fin de la partie")


In [7]:
import re

# Split the clean data by paragraphes
paragraphs = re.split(r'\n(?=[A-Z]{5})', text_clean.strip(), flags=re.MULTILINE)

the split generate 5 paragraphes

In [8]:
print(paragraphs)

["Règles du jeu officielles du Monopoly®  \n \nGaranties presque sans erreurs   \nBUT DU JEU  :  \nEtre le dernier joueur à rester en jeu, c'est -à-dire le dernier \njoueur n'ayant pas fait faillite.  \n ", 'PREPARATION  \n1. Prenez le casier de la Banque et disposez dans leurs emplacements les Maisons, les Hôtels, les Titre s de \nPropriété et les billets (classés selon leur valeur).   \n2. La première fois que vous jouez, détachez les pieds en caoutchouc et disposez -les sous les coins des \nsabots.  \nDisposez les sabots à l\'endroit prévu sur le plateau de jeu.   \n3. Détachez les cartes Chanc e et Caisse de Communauté, mélangez -les et placez -les face cachée dans les \nsabots correspondants.   \n4. Chaque joueur choisit un pion et le place sur la case "Départ".  \n5. Le Banquier et la Banque.  \nChoisissez comme Banquier un joueur. S\'il y a plus de 5 joueurs, l \'un des joueurs peut décider de tenir \nuniquement le rôle de Banquier.  \nLe Banquier remet à chaque joueur une somme

# TOKENIZE



In [9]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources if not already downloaded
nltk.download('punkt')

parag_tokens = []
# Tokenize the text into words
for paragraph in paragraphs :
  tokens = word_tokenize(paragraph)
  parag_tokens.append(tokens)

# Print the tokens
print(tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['PARTIE', 'LIMITEE', 'DANS', 'LE', 'TEMPS', 'Pour', 'cette', 'variante', "d'une", 'partie', 'courte', ',', 'les', 'joueurs', 'décident', ',', 'avant', 'de', 'co', 'mmencer', ',', 'de', "l'heure", 'à', 'laquelle', 'la', 'partie', 'doit', 'finir', '.', 'Le', 'joueur', 'le', 'plus', 'riche', 'sera', 'déclaré', 'vainqueur', '.', 'Au', 'début', 'de', 'la', 'partie', ',', 'le', 'Banquier', 'mélange', 'tous', 'les', 'Titres', 'de', 'Propriété', 'et', 'en', 'distribue', 'deux', 'à', 'chaque', 'joueur', ',', 'face', 'cachée', '.', 'Chacun', 'paye', 'immédiatement', 'à', 'la', 'B', 'anque', 'le', 'prix', 'des', 'deux', 'propriétés', "qu'il", 'a', 'reçues', '.', 'Le', 'jeu', 'se', 'déroule', 'normalement', 'jusqu', "'", 'à', "l'heure", 'préalablement', 'fixée', '.', 'On', 'arrête', 'alors', 'toute', 'opération', '(', 'si', "l'un", 'des', 'joueurs', 'avait', 'commencé', 'à', 'jouer', ',', 'il', 'peut', 'terminer', 'son', 'tour', 'de', 'jeu', ')', '.', 'Chaque', 'joueur', 'évalue', 'alors', "l'ens

# VECTORISATION

In [10]:
from sentence_transformers import SentenceTransformer

def paragraphs_to_vectors(tokenized_paragraphs):
    model = SentenceTransformer('paraphrase-distilroberta-base-v1')
    paragraph_vectors = []
    for paragraph in tokenized_paragraphs:
        # Flatten list of sentences into a single string
        paragraph_text = ' '.join([' '.join(word) for word in paragraph])
        # Encode the paragraph into a vector
        paragraph_vector = model.encode(paragraph_text)
        paragraph_vectors.append(paragraph_vector)
    return paragraph_vectors



In [11]:
paragraph_vectors = paragraphs_to_vectors(parag_tokens)

# Print the vectors
for i, vector in enumerate(paragraph_vectors):
    print(f"Vector for Paragraph {i+1}: {vector}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector for Paragraph 1: [ 2.72751540e-01 -2.88461745e-01 -7.65542984e-02 -2.65171468e-01
 -7.93549657e-01 -2.20171228e-01  2.69664526e-01  1.89608485e-01
 -6.03106737e-01 -3.46582413e-01 -3.28532219e-01  4.96502638e-01
  6.57373816e-02 -5.82794324e-02  7.03376710e-01 -1.71905577e-01
  6.88432753e-02  1.14339089e+00 -4.12644148e-01 -4.33890879e-01
 -1.19948626e-01  1.02485716e-02 -4.84386861e-01  7.45596886e-01
 -5.39320052e-01 -2.17458487e-01 -5.57938814e-01  5.50425053e-01
 -2.75863349e-01  2.33437777e-01  8.26712698e-02 -5.78483224e-01
 -2.46203035e-01  3.01215112e-01 -2.86019947e-02 -6.06594265e-01
  7.88466811e-01  5.46062738e-02  1.07510239e-01  4.31851089e-01
 -2.66601175e-01 -6.18987381e-01  1.67744622e-01  3.14919233e-01
  2.71469682e-01  2.34134316e-01  6.05115712e-01  1.29189861e+00
 -1.84165359e-01  4.59120631e-01  7.33957738e-02 -4.78935868e-01
 -2.05411762e-01 -1.50619030e-01  4.32422087e-02  8.65352988e-01
  5.85453749e-01 -7.21879065e-01  1.81034327e-01 -9.13004875e-02
 

# BERT EMBEDDINGS

In [18]:
!pip install faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 41.2 MB/s eta 0:00:00


In [21]:
from transformers import BertTokenizer, BertModel
import torch
import faiss
import numpy as np

In [48]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.pooler_output
    return embeddings.numpy()

In [49]:
# Generate embeddings for each document
embeddings_list = [get_bert_embeddings(doc) for doc in paragraphs]
embeddings = np.concatenate(embeddings_list)

In [50]:
# Store embeddings in a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance metric
index.add(embeddings)

In [57]:
from transformers import BertTokenizer, BertModel, GPT2LMHeadModel, GPT2Tokenizer
import torch
import faiss
import numpy as np
documents = paragraphs

# Load BERT model and tokenizer for document retrieval
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load GPT-2 model and tokenizer for answer generation
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

# Load preprocessed document embeddings and documents list
# Assuming you have 'embeddings.npy' containing embeddings and 'documents.txt' containing corresponding documents


# Function to retrieve relevant documents for a query
def retrieve_documents(query_text, k=5):
    query_embedding = get_query_embedding(query_text)
    D, I = index.search(query_embedding, k)
    return [documents[i] for i in I[0]]

# Function to generate BERT embeddings for query
def get_query_embedding(query_text):
    inputs = bert_tokenizer(query_text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    query_embedding = outputs.pooler_output.numpy()
    return query_embedding

# Function to generate an appropriate answer based on retrieved documents
def generate_answer(query_text):
    relevant_documents = retrieve_documents(query_text)
    concatenated_text = ". ".join(relevant_documents)  # Concatenate relevant documents
    input_ids = gpt_tokenizer.encode("Q: " + query_text + " A: " + concatenated_text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        output = gpt_model.generate(input_ids, max_length=520, num_return_sequences=1, temperature=0.7)
    answer = gpt_tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# Example query
query_text = "Etre le dernier joueur à"
answer = generate_answer(query_text)
print("Answer:", answer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Q: Etre le dernier joueur à A: Règles du jeu officielles du Monopoly®  
 
Garanties presque sans erreurs   
BUT DU JEU  :  
Etre le dernier joueur à rester en jeu, c'est -à-dire le dernier 
joueur n'ayant pas fait faillite.  
. DEROULEMENT DU JEU   
>  
Quand arrive votre tour de jeu, lancez les 2 dés puis avancez vot re pion sur le plateau de jeu, d'autant de cases 
que l'indique le total des dés, et dans le sens indiqué par la flèche. La case sur laquelle vous allez vous arrêter 
vous indique ce que vous devez ou pouvez faire. Deux pions ou plus peuvent se trouver sur u ne même case au 
même moment. Suivant la nature de la case sur laquelle vous êtes arrivé, vous pouvez effectuer l'une des 
opérations suivantes :  
 acheter un terrain ou une autre propriété,  
 payer un loyer (si la propriété appartient à un autre joueur),  
 payer des impôts,  
 tirer une carte "Chance" ou "Caisse de Communauté",  
 aller en Prison,  
 bénéficier du Parc Gratuit,  
 toucher un salaire de 20.

In [56]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")
import torch
import faiss
import numpy as np
documents = paragraphs

# Load BERT model and tokenizer for document retrieval
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Load GPT-2 model and tokenizer for answer generation
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

# Load preprocessed document embeddings and documents list
# Assuming you have 'embeddings.npy' containing embeddings and 'documents.txt' containing corresponding documents


# Function to retrieve relevant documents for a query
def retrieve_documents(query_text, k=5):
    query_embedding = get_query_embedding(query_text)
    D, I = index.search(query_embedding, k)
    return [documents[i] for i in I[0]]

# Function to generate BERT embeddings for query
def get_query_embedding(query_text):
    inputs = bert_tokenizer(query_text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    query_embedding = outputs.pooler_output.numpy()
    return query_embedding

# Function to generate an appropriate answer based on retrieved documents
def generate_answer(query_text):
    relevant_documents = retrieve_documents(query_text)
    concatenated_text = ". ".join(relevant_documents)  # Concatenate relevant documents
    input_ids = gpt_tokenizer.encode("Q: " + query_text + " A: " + concatenated_text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        output = gpt_model.generate(input_ids, max_length=520, num_return_sequences=1, temperature=0.7)
    answer = gpt_tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

# Example query
query_text = "Etre le dernier joueur à"
answer = generate_answer(query_text)
print("Answer:", answer)


["Règles du jeu officielles du Monopoly®  \n \nGaranties presque sans erreurs   \nBUT DU JEU  :  \nEtre le dernier joueur à rester en jeu, c'est -à-dire le dernier \njoueur n'ayant pas fait faillite.  \n ", 'PREPARATION  \n1. Prenez le casier de la Banque et disposez dans leurs emplacements les Maisons, les Hôtels, les Titre s de \nPropriété et les billets (classés selon leur valeur).   \n2. La première fois que vous jouez, détachez les pieds en caoutchouc et disposez -les sous les coins des \nsabots.  \nDisposez les sabots à l\'endroit prévu sur le plateau de jeu.   \n3. Détachez les cartes Chanc e et Caisse de Communauté, mélangez -les et placez -les face cachée dans les \nsabots correspondants.   \n4. Chaque joueur choisit un pion et le place sur la case "Départ".  \n5. Le Banquier et la Banque.  \nChoisissez comme Banquier un joueur. S\'il y a plus de 5 joueurs, l \'un des joueurs peut décider de tenir \nuniquement le rôle de Banquier.  \nLe Banquier remet à chaque joueur une somme

# LANGCHAIN

In [41]:
%pip install --upgrade --quiet  langchain sentence_transformers

In [42]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [43]:
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [44]:
query_result = embeddings.embed_query(text)

In [45]:
query_result[:3]

[-0.010269589722156525, -0.052917081862688065, -0.00548364594578743]